In [36]:
%config IPCompleter.greedy=True 
import dicom #read the dicom files
import os #do directory operations
import pandas as pd #nice for data analysis

data_dir = '/Volumes/USB_DRIVE/Data Science Bowl/stage1/stage1/' #directory for the data
patients = os.listdir(data_dir) # get all items inside of folder
labels_df = pd.read_csv('stage1_labels.csv', index_col=0) # put into dataframe

labels_df.head()

,cancer
id,
0015ceb851d7251b8f399e39779d1e7d,1
0030a160d58723ff36d73f41b170ec21,0
003f41c78e6acfa92430a057ac0b306e,0
006b96310a37b36cccb2ab48d10b49a3,1
008464bb8521d09a42985dd8add3d0d2,1


In [44]:
%config IPCompleter.greedy=True 
for patient in patients[:1]: # this is making a list containing only the first patient
    label = labels_df.get_value(patient, 'cancer')
    path = data_dir + patient # path to dicom files for patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)] # reading in all the dicom files for patient
    #x is the dicom file
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2])) # This sorts the dicom files by the third value in image position patient attribute
    print(len(slices), slices[0].pixel_array.shape)
#     print(slices[0]) # this is the dicom file for the patient

195 (512, 512)


In [45]:
len(patients)

1595